In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import duckdb
import gc

In [ ]:

#!pip install duckdb -q

import duckdb
import pandas as pd
import gc


con = duckdb.connect()


d = '/content/drive/MyDrive/datos123/train.csv'

print(" Exploración del data set")
print("="*60)

# 2. VER LAS PRIMERAS FILAS
print(" 5 FILAS:")
print("="*60)
head = con.execute(f"""
    SELECT *
    FROM '{d}'
    LIMIT 5
""").df()
print(head)

# 3. CONTAR TOTAL DE REGISTROS
print("\nINFORMACIÓN GENERAL:")
print("="*60)
total_rows = con.execute(f"""
    SELECT COUNT(*) as total
    FROM '{d}'
""").fetchone()[0]
print(f"Total de registros: {total_rows:,}")

# 4. DISTRIBUCIÓN DE LA VARIABLE OBJETIVO
print("\nDISTRIBUCIÓN DE is_attributed:")
print("="*60)
distribution = con.execute(f"""
    SELECT
        is_attributed,
        COUNT(*) as count,
        ROUND(COUNT(*) * 100.0 / {total_rows}, 4) as percentage
    FROM '{d}'
    GROUP BY is_attributed
    ORDER BY is_attributed
""").df()
print(distribution)

positivos = int(distribution[distribution['is_attributed']==1]['count'].iloc[0])
negativos = int(distribution[distribution['is_attributed']==0]['count'].iloc[0])

print(f"\n Positivos (is_attributed=1): {positivos:,}")
print(f" Negativos (is_attributed=0): {negativos:,}")
print(f"Desbalance: 1 positivo cada {negativos//positivos:,} negativos")

# 5. DISTRIBUCIÓN POR OTRAS COLUMNAS CLAVE
print("\nTOP 10 APPS MÁS FRECUENTES:")
print("="*60)
top_apps = con.execute(f"""
    SELECT
        app,
        COUNT(*) as clicks,
        SUM(is_attributed) as conversiones,
        ROUND(SUM(is_attributed) * 100.0 / COUNT(*), 2) as conversion_rate
    FROM '{d}'
    GROUP BY app
    ORDER BY clicks DESC
    LIMIT 10
""").df()
print(top_apps)

print("\n TOP 10 CHANNELS MÁS FRECUENTES:")
print("="*60)
top_channels = con.execute(f"""
    SELECT
        channel,
        COUNT(*) as clicks,
        SUM(is_attributed) as conversiones,
        ROUND(SUM(is_attributed) * 100.0 / COUNT(*), 2) as conversion_rate
    FROM '{d}'
    GROUP BY channel
    ORDER BY clicks DESC
    LIMIT 10
""").df()
print(top_channels)

# 6. RANGO DE FECHAS
print("\n RANGO TEMPORAL:")
print("="*60)
time_range = con.execute(f"""
    SELECT
        MIN(click_time) as fecha_inicio,
        MAX(click_time) as fecha_fin
    FROM '{d}'
""").df()
print(time_range)

print("\n" + "="*60)
print(" EXPLORACIÓN COMPLETADA")
print("="*60)


 Exploración del data set
 5 FILAS:
       ip  app  device  os  channel          click_time attributed_time  \
0   83230    3       1  13      379 2017-11-06 14:32:21             NaT   
1   17357    3       1  19      379 2017-11-06 14:33:34             NaT   
2   35810    3       1  13      379 2017-11-06 14:34:12             NaT   
3   45745   14       1  13      478 2017-11-06 14:34:52             NaT   
4  161007    3       1  13      379 2017-11-06 14:35:08             NaT   

   is_attributed  
0              0  
1              0  
2              0  
3              0  
4              0  

INFORMACIÓN GENERAL:


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Total de registros: 184,903,890

DISTRIBUCIÓN DE is_attributed:


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

   is_attributed      count  percentage
0              0  184447044     99.7529
1              1     456846      0.2471

 Positivos (is_attributed=1): 456,846
 Negativos (is_attributed=0): 184,447,044
Desbalance: 1 positivo cada 403 negativos

TOP 10 APPS MÁS FRECUENTES:


...

   app    clicks  conversiones  conversion_rate
0    3  33911780       10261.0             0.03
1   12  24179003        2640.0             0.01
2    2  21642136        5661.0             0.03
3    9  16458268       18823.0             0.11
4   15  15958970        3418.0             0.02
5   18  15756587        7919.0             0.05
6   14  10027169        2505.0             0.02
7    1   5796274        1230.0             0.02
8   13   4329409         721.0             0.02
9    8   3731948        6875.0             0.18

 TOP 10 CHANNELS MÁS FRECUENTES:


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

   channel    clicks  conversiones  conversion_rate
0      280  15065927        4075.0             0.03
1      245   8873025         829.0             0.01
2      107   8457471        4262.0             0.05
3      477   7188340        1212.0             0.02
4      134   5924250        3524.0             0.06
5      259   5787004        2558.0             0.04
6      153   5537977         765.0             0.01
7      265   5446388         732.0             0.01
8      178   5271408         627.0             0.01
9      121   4692438        2055.0             0.04

 RANGO TEMPORAL:


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

         fecha_inicio           fecha_fin
0 2017-11-06 14:32:21 2017-11-09 16:00:00

 EXPLORACIÓN COMPLETADA


| Ítem | Resultado / Valor | Interpretación |
|------|-------------------|--------------------|
| **Registros totales** | 184,903,890 | Total data ser |
| **Positivos (conversiones)** | 456,846 (0.247%) | Muy pocos casos donde sí hubo descarga |
| **Negativos (no convierten)** | 184,447,044 (99.75%) | Casi todos los clicks no terminan en nada |
| **Desbalance** | 1 : 403 | Por cada descarga hay 403 clicks inútiles |
| **Inicio del registro** | 06/11/2017 14:32 | inicio |
| **Fin del registro** | 09/11/2017 16:00 | tiempo de recolección |
| **Duración total** | ~3 días | Tiempo de todo el data set |
| **Tráfico diario** | ~62 millones/día | trafico de los datos  |
| **Bots / clicks falsos** | Probable | Ese volumen suele venir de robots o granjas de clicks |
| **Relevancia del tiempo** | Hora/día importa | La hora puede ayudar a encontrar patrones raros |




In [ ]:
RATIO = 16
positivos = 456846
negativos = positivos * RATIO
total = positivos + negativos

print(f"Ratio: 1:{RATIO}")
print(f"Positivos: {positivos:,}")
print(f"Negativos: {negativos:,}")
print(f"TOTAL: {total:,}")
print("Creando muestra...")

sample = con.execute(f"""
    SELECT * FROM (
        SELECT * FROM '{d}' WHERE is_attributed = 1
        UNION ALL
        SELECT * FROM '{d}'
        WHERE is_attributed = 0
        USING SAMPLE reservoir({negativos} ROWS)
    )
    ORDER BY RANDOM()
""").df()

print(f"Datos: {len(sample):,} registros")
print(f"Positivos: {(sample['is_attributed']==1).sum():,}")
print(f"Negativos: {(sample['is_attributed']==0).sum():,}")

sample.to_parquet('/content/drive/MyDrive/datos123/datos8M.parquet', index=False)


Ratio: 1:16
Positivos: 456,846
Negativos: 7,309,536
TOTAL: 7,766,382
Creando muestra...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Datos: 7,751,745 registros
Positivos: 456,846
Negativos: 7,294,899


| Parte | Qué hace | Explicación|
|-------|----------|-------------------|
| Definición del ratio | `RATIO = 16`<br>`negativos = positivos * 16` | Se decide tomar 16 negativos por cada 1 positivo para equilibrar el dataset. |
| Cálculo previo | Positivos = 456,846<br>Negativos = 7,309,536<br>Total = 7,766,382 | Es un cálculo teórico para saber cuántos negativos hay que muestrear. |
| Selección de datos (SQL) | `SELECT * FROM d WHERE is_attributed = 1` | Se seleccionan **todos** los registros positivos completos. |
| Muestreo de negativos | `SELECT ... USING SAMPLE reservoir(negativos ROWS)` | Se toma solo una **muestra** de negativos usando muestreo tipo *reservoir* hasta aproximarse al número deseado. |
| Unión de ambos | `UNION ALL` | Junta los positivos completos + la muestra de negativos, sin eliminarlos. |
| Mezcla aleatoria | `ORDER BY RANDOM()` | Desordena todo para que positivos y negativos queden mezclados y no en bloques. |
| Resultado final | Datos finales = 7,751,745<br>Positivos = 456,846<br>Negativos = 7,294,899 | El muestreo no da exactamente los 7,309,536 negativos teóricos, pero queda muy cercano al ratio 1:16. |
| Conclusión | Dataset balanceado por submuestreo de negativos | Se reduce el desbalance manteniendo todos los positivos para un mejor entrenamiento. |


In [ ]:
muestra = pd.read_parquet("/content/drive/MyDrive/datos123/datos8M.parquet")


In [ ]:
print("Verificando muestra actual...")
print(f"\nTotal registros: {len(muestra):,}")
print(f"\nDistribucion:")
print(muestra['is_attributed'].value_counts())
print(f"\nBalance: {muestra['is_attributed'].mean()*100:.2f}% positivos")

print("\nProporciones por APP (top 5):")
print(muestra.groupby('app')['is_attributed'].agg(['count', 'sum', 'mean']).sort_values('count', ascending=False).head())

print("\nProporciones por CHANNEL (top 5):")
print(muestra.groupby('channel')['is_attributed'].agg(['count', 'sum', 'mean']).sort_values('count', ascending=False).head())

Verificando muestra actual...

Total registros: 7,751,745

Distribucion:
is_attributed
0    7294899
1     456846
Name: count, dtype: int64

Balance: 5.89% positivos

Proporciones por APP (top 5):
      count    sum      mean
app                         
12   978261   2640  0.002699
3    927844  10261  0.011059
2    919633   5661  0.006156
15   854299   3418  0.004001
9    695975  18823  0.027046

Proporciones por CHANNEL (top 5):
          count   sum      mean
channel                        
245      618129   829  0.001341
134      398889  3524  0.008835
259      346559  2558  0.007381
477      328748  1212  0.003687
121      298296  2055  0.006889


In [ ]:
import duckdb

con = duckdb.connect()

# Registrar tu DataFrame 'muestra' en DuckDB
con.register('muestra_temp', muestra)

RATIO = 16
positivos_total = 456846
negativos_objetivo = positivos_total * RATIO

print("Estratificando la muestra actual...")

muestra_estratificada = con.execute(f"""
    WITH todos_positivos AS (
        SELECT * FROM muestra_temp WHERE is_attributed = 1
    ),
    negativos_por_estrato AS (
        SELECT
            *,
            ROW_NUMBER() OVER (
                PARTITION BY app, channel
                ORDER BY RANDOM()
            ) as rn,
            FLOOR(
                COUNT(*) OVER (PARTITION BY app, channel)
                * {negativos_objetivo / 7294899.0}
            ) as sample_size
        FROM muestra_temp
        WHERE is_attributed = 0
    ),
    negativos_estratificados AS (
        SELECT * EXCLUDE(rn, sample_size)
        FROM negativos_por_estrato
        WHERE rn <= GREATEST(sample_size, 1)
    ),
    union_final AS (
        SELECT * FROM todos_positivos
        UNION ALL
        SELECT * FROM negativos_estratificados
    )
    SELECT * FROM union_final
    ORDER BY RANDOM()
""").df()

print(f"\nMuestra estratificada: {len(muestra_estratificada):,}")
print(f"Positivos: {(muestra_estratificada['is_attributed']==1).sum():,}")
print(f"Negativos: {(muestra_estratificada['is_attributed']==0).sum():,}")

# Guardar
ruta = '/content/drive/MyDrive/datos123/estratificada.parquet'
muestra_estratificada.to_parquet(ruta, index=False)
print(f"\nGuardado: {ruta}")


Estratificando la muestra actual...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


Muestra estratificada: 7,751,745
Positivos: 456,846
Negativos: 7,294,899

Guardado: /content/drive/MyDrive/datos123/estratificada.parquet
